<a href="https://colab.research.google.com/github/USAFADFCS/final-project-jon-and-ari/blob/main/Ari_Jon_AI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -U transformers

In [10]:
# asr_whisper.py
import os, torch
from transformers import pipeline

MODEL_ID = "openai/whisper-large-v3-turbo"
DEVICE = 0 if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

# Build one reusable pipeline
asr = pipeline(
    "automatic-speech-recognition",
    model=MODEL_ID,
    device=DEVICE,
    torch_dtype=DTYPE,
    chunk_length_s=30,           # robust for long audio
    stride_length_s=5,           # overlap for context
    return_timestamps=True
)

MEDICAL_LEXICON = (
    "tourniquet, hemorrhage, capillary refill, obey commands, airway, "
    "respirations, pulse, radial pulse, naloxone, unresponsive, shock"
)

def transcribe(path: str) -> dict:
    return asr(
        path,
        generate_kwargs={
            "task": "transcribe",      # or "translate" if needed
            "temperature": 0.0,
            "num_beams": 5
        },
        # primes decoding with triage vocabulary
        prompt=MEDICAL_LEXICON,
        return_timestamps=True
    )


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [11]:
# vad.py
import torchaudio

def simple_vad_chunks(wav_path, min_speech_len=0.6):
    wav, sr = torchaudio.load(wav_path)
    wav = torchaudio.functional.resample(wav, sr, 16000)
    vad = torchaudio.transforms.Vad(sample_rate=16000)
    voiced = vad(wav.squeeze(0))
    # Fallback: if overly aggressive, just return original path
    if voiced.numel() < 16000 * min_speech_len:
        return [wav_path]
    # For brevity, write voiced chunk to temp file; in production, slice windows
    out = "/tmp/voiced.wav"
    torchaudio.save(out, voiced.unsqueeze(0), 16000)
    return [out]


In [13]:
%%writefile asr_whisper.py
import os, torch
from transformers import pipeline

MODEL_ID = "openai/whisper-large-v3-turbo"
DEVICE = 0 if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if torch.cuda.is_available() else torch.float32

# Build one reusable pipeline
asr = pipeline(
    "automatic-speech-recognition",
    model=MODEL_ID,
    device=DEVICE,
    torch_dtype=DTYPE,
    chunk_length_s=30,           # robust for long audio
    stride_length_s=5,           # overlap for context
    return_timestamps=True
)

MEDICAL_LEXICON = (
    "tourniquet, hemorrhage, capillary refill, obey commands, airway, "
    "respirations, pulse, radial pulse, naloxone, unresponsive, shock"
)

def transcribe(path: str) -> dict:
    return asr(
        path,
        generate_kwargs={
            "task": "transcribe",      # or "translate" if needed
            "temperature": 0.0,
            "num_beams": 5
        },
        # primes decoding with triage vocabulary
        prompt=MEDICAL_LEXICON,
        return_timestamps=True
    )

Writing asr_whisper.py


In [14]:
%%writefile vad.py
import torchaudio

def simple_vad_chunks(wav_path, min_speech_len=0.6):
    wav, sr = torchaudio.load(wav_path)
    wav = torchaudio.functional.resample(wav, sr, 16000)
    vad = torchaudio.transforms.Vad(sample_rate=16000)
    voiced = vad(wav.squeeze(0))
    # Fallback: if overly aggressive, just return original path
    if voiced.numel() < 16000 * min_speech_len:
        return [wav_path]
    # For brevity, write voiced chunk to temp file; in production, slice windows
    out = "/tmp/voiced.wav"
    torchaudio.save(out, voiced.unsqueeze(0), 16000)
    return [out]

Writing vad.py


In [15]:
# main.py
from asr_whisper import transcribe
from vad import simple_vad_chunks

def transcribe_with_vad(path):
    out = {"text": "", "segments": []}
    for chunk in simple_vad_chunks(path):
        r = transcribe(chunk)
        out["text"] += (" " + r["text"]).strip()
        if "chunks" in r:
            out["segments"].extend(r["chunks"])
    return out

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [17]:
TRIAGE_SCHEMA = {
  "patient_id": str,
  "entities": {
    "bleeding_severe": bool,
    "can_walk": bool | None,
    "obeys_commands": bool | None,
    "resp_rate": int | None,
    "cap_refill_sec": float | None,
    "mental_status": str | None
  },
  "evidence": list,              # text snippets / timestamps
  "triage_candidate": str,       # Immediate | Delayed | Minimal | Expectant | Unknown
  "uncertainty": float,
  "next_question": str | None
}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')